In [1]:
%matplotlib widget
import caesar
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import scipy 

/Users/ondrea/anaconda3/lib/python3.8/_collections_abc.py:666: MatplotlibDeprecationWarning: The global colormaps dictionary is no longer considered public API.
  self[key]


In [2]:
obj = caesar.load('/Users/ondrea/Downloads/caesar_296.hdf5') # 296=redshift0

yt : [INFO     ] 2021-08-30 12:00:57,229 Opening /Users/ondrea/Downloads/caesar_296.hdf5
yt : [INFO     ] 2021-08-30 12:00:57,419 Found 54135 halos
yt : [INFO     ] 2021-08-30 12:00:57,426 Found 6480 galaxies


In [3]:
#obj.galaxies[1].masses

In [4]:
orig_index = np.arange(0,len(obj.galaxies))
sfr = np.array([gal.sfr for gal in obj.galaxies])
stellar_masses = np.array([gal.masses['stellar'] for gal in obj.galaxies])
bh_masses = np.array([gal.masses['bh'] for gal in obj.galaxies])
gas_masses = np.array([gal.masses['gas'] for gal in obj.galaxies])
metallicity = np.array([gal.metallicities['stellar'] for gal in obj.galaxies])                    
temp = obj.galaxies[1].temperatures['mass_weighted']
radius = np.array([gal.radii['stellar_r20'] for gal in obj.galaxies])


In [5]:
sfr/stellar_masses

array([0., 0., 0., ..., 0., 0., 0.])

In [6]:
bh_masses[bh_masses == 0] = 1e-99
stellar_masses[stellar_masses == 0] = 1e-99
gas_masses[gas_masses == 0] = 1e-99
metallicity[metallicity == 0] = 1e-99

In [7]:
#obj.galaxies[1].virial_quantities

In [8]:
#obj.galaxies[1].radii

In [9]:
gal_data = pd.DataFrame({'ID': orig_index, 'bh_masses': np.log10(bh_masses),\
                         'stellar_masses': np.log10(stellar_masses), 'sfr': sfr, 'gas_masses' : np.log10(gas_masses),\
                         'metallicity' : np.log10(metallicity), 'radius' : radius})

In [10]:
#What SFRs do we have?
gal_data['sfr'].value_counts()

0.000000    1015
0.195986       1
0.078754       1
0.878105       1
0.034745       1
            ... 
0.158136       1
0.638304       1
1.068277       1
0.933351       1
0.815047       1
Name: sfr, Length: 5466, dtype: int64

In [11]:
#Make a col of specific star formation rate (SFR/gal stellar mass)*(1e9 Gyr/yr)
gal_data["ssfr"] = (sfr/stellar_masses)*1e9 

In [12]:
stellar_masses

array([1.05960459e+12, 8.26104807e+11, 7.95258585e+11, ...,
       2.64359616e+08, 2.61941984e+08, 2.61884624e+08])

In [13]:
gal_data['sfr']

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
6475    0.0
6476    0.0
6477    0.0
6478    0.0
6479    0.0
Name: sfr, Length: 6480, dtype: float64

In [14]:
gal_data

,ID,bh_masses,stellar_masses,sfr,gas_masses,metallicity,radius,ssfr
0,0,10.290369,12.025144,0.0,-99.0,-1.709673,12.585268,0.0
1,1,9.681071,11.917035,0.0,-99.0,-1.686568,7.108331,0.0
2,2,9.890138,11.900508,0.0,-99.0,-1.713443,8.093110,0.0
3,3,9.811176,11.867450,0.0,-99.0,-1.702289,7.462501,0.0
4,4,9.830744,11.821318,0.0,-99.0,-1.691337,7.025398,0.0
...,...,...,...,...,...,...,...,...
6475,6475,-99.000000,8.431755,0.0,-99.0,-2.948695,2.281374,0.0
6476,6476,-99.000000,8.423798,0.0,-99.0,-2.864811,2.289696,0.0
6477,6477,-99.000000,8.422195,0.0,-99.0,-3.026400,1.733394,0.0
6478,6478,-99.000000,8.418205,0.0,-99.0,-2.740298,2.117305,0.0


In [15]:
pd.plotting.scatter_matrix(gal_data, alpha=0.2, figsize=(10, 10), diagonal="kde");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [130]:
ifig=1;plt.close(ifig);plt.figure(ifig) 
plt.scatter(gal_data.stellar_masses, np.log10(gal_data.ssfr), s=2, alpha=0.5);
plt.xlabel('log stellar mass')
plt.ylabel('log ssfr')
plt.ylim(-3,2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/Users/ondrea/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


(-3.0, 2.0)

In [120]:
from sklearn.mixture import GaussianMixture

In [154]:
x = gal_data['ssfr'].to_numpy()
y = gal_data['stellar_masses'].to_numpy()

In [236]:
X = np.zeros([len(x), 2])
for i in range(len(x)):
    X[i,0] = x[i]
    X[i,1] = y[i]

In [244]:
X

array([0., 0., 0., ..., 0., 0., 0.])

In [301]:
gmm = GaussianMixture(n_components=2, covariance_type='diag',random_state=1).fit(X)
labels = gmm.predict(X)

In [302]:
GaussianMixture?

In [303]:
#labels

In [304]:
ifig=2;plt.close(ifig);plt.figure(ifig)
#plt.scatter(gal_data.stellar_masses[labels == 1], np.log10(gal_data.ssfr[labels == 1]), s=4, alpha=0.4, c='b');
#plt.scatter(gal_data.stellar_masses[labels == 0], np.log10(gal_data.ssfr[labels == 0]), s=4, alpha=0.4, c='k');
plt.scatter(X[:, 1], np.log10(X[:, 0]), c=labels, s=4, cmap='viridis');
plt.xlabel('log stellar mass')
plt.ylabel('log ssfr')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-304-cc018e5add35>:4: RuntimeWarning: divide by zero encountered in log10
  plt.scatter(X[:, 1], np.log10(X[:, 0]), c=labels, s=4, cmap='viridis');


Text(0, 0.5, 'log ssfr')

In [231]:
# Generate some data
from sklearn.datasets import make_blobs
X, y_true = make_blobs(n_samples=400, centers=4,
                       cluster_std=0.60, random_state=0)
X = X[:, ::-1] # flip axes for better plotting

In [232]:
X

array([[ 6.48826703e+00,  2.50064460e-01],
       [ 2.52194722e+00, -2.69798647e+00],
       [ 3.06877291e+00, -2.90007604e+00],
       [ 4.31427482e+00,  7.02950576e-01],
       [ 2.38449945e+00, -2.53563494e+00],
       [ 7.80540956e+00, -1.70297829e+00],
       [ 8.38773426e-01,  2.22322228e+00],
       [ 4.17614316e+00,  7.93137001e-03],
       [ 3.02668532e+00, -1.79041773e+00],
       [ 1.26987864e+00,  1.76747403e+00],
       [ 7.81974467e+00, -9.49424332e-01],
       [ 1.87335276e-01,  2.51921931e+00],
       [ 3.43116792e+00,  2.33812285e+00],
       [ 3.39806103e+00, -2.39482483e+00],
       [ 2.95679593e+00, -7.91981675e-01],
       [ 1.13171965e+00,  3.22881491e+00],
       [ 1.59701749e+00, -1.80822253e+00],
       [ 4.09867867e-01,  1.62350489e+00],
       [ 3.89290127e+00,  1.21767506e+00],
       [ 3.54666555e+00, -1.22768985e+00],
       [ 3.53554556e+00, -1.06406044e+00],
       [ 2.66329169e+00, -2.07216196e+00],
       [ 7.15134224e+00, -1.95483315e+00],
       [ 4.